In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://m.imdb.com/chart/top/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
response = requests.get(url , headers = headers)

soup = BeautifulSoup(response.content , 'html.parser') 
content = soup.prettify()
#print("\n * \n" + content)

if response.status_code==200:
    print("access successfull!")
else :
    print(f"{response.status_code} error occured!")

#print(content)
#with open("imdb.txt" , "w") as file:
#    file.write(content)

access successfull!


In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import re

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "en-US,en;q=0.9"
}
url = "https://m.imdb.com/chart/top/"


resp = requests.get(url, headers=headers, timeout=10)
if resp.status_code != 200:
    raise SystemExit(f"Failed to fetch {url}: {resp.status_code}")

soup = BeautifulSoup(resp.content , "html.parser")


movie_list_parent = soup.find("ul", class_=re.compile(r"ipc-metadata-list.*"))
if not movie_list_parent:
    # Save HTML for inspection and raise a helpful error
    with open("imdb_debug.html","w", encoding="utf-8") as f:
        f.write(soup.prettify())
    raise SystemExit("Could not find the movie list. Saved 'imdb_debug.html' for inspection.")

movie_list = movie_list_parent.find_all("li", class_ ="ipc-metadata-list-summary-item")

with open("goat.csv" , "w", newline="", encoding="utf-8") as file:
    writer=csv.writer(file)
    
    count = 0
    for movie in movie_list:
        if count >= 5:
            break

        # Extract movie name and rank
        title_element = movie.find("h3",class_ ="ipc-title__text")
        if title_element:
            full_title_text = title_element.get_text(strip=True)
            movie_rank = full_title_text.split(".")[0] if "." in full_title_text else ""
            movie_name = ".".join(full_title_text.split(".")[1:]).strip() if "." in full_title_text else full_title_text
        else:
            movie_rank = ""
            movie_name = ""


        year_element = movie.find("span", class_ ="sc-b4f120f6-7 hoOxkw cli-title-metadata-item")
        movie_year = year_element.get_text().strip() if year_element else ""

        rating_element = movie.find("span" , class_="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")
        movie_rating = rating_element.text.split()[0] if rating_element else ""

        link_element = movie.find("a" , class_ = "ipc-title-link-wrapper")
        summary_link = link_element.get("href") if link_element else ""

        summary = ""
        if summary_link: # Only attempt to get summary if link exists
            try:
                summary_linked_page = requests.get("https://m.imdb.com"+summary_link ,headers=headers, timeout=10)
                if summary_linked_page.status_code == 200:
                    soup2 = BeautifulSoup( summary_linked_page.content , "html.parser")
                    summary_element = soup2.find('div',class_='ipc-html-content-inner-div')
                    summary = summary_element.get_text(strip=True) if summary_element else ""
            except Exception:
                summary = ""

        writer.writerow([movie_name ,movie_year , movie_rating , summary])
        count += 1

In [3]:
import pandas as pd

df_movies = pd.read_csv('goat.csv', header=None, names=['Title', 'Year', 'Rating', 'Summary'])
display(df_movies)

,Title,Year,Rating,Summary
0,The Shawshank Redemption,1994,9.3,Reviewers say 'The Shawshank Redemption' is ce...
1,The Godfather,1972,9.2,Reviewers say 'The Godfather' is acclaimed for...
2,The Dark Knight,2008,9.1,Reviewers say 'The Dark Knight' is celebrated ...
3,The Godfather Part II,1974,9.0,Reviewers say 'The Godfather Part II' is accla...
4,12 Angry Men,1957,9.0,Reviewers say '12 Angry Men' is acclaimed for ...
